In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.sparse
from scipy.sparse import *
from scipy.sparse import linalg
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import collections
import os, time, collections, shutil
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### loading data

In [3]:
names = ['squareId','timeInterval','countryCode','smsIn','smsOut','callIn','callOut','Internet']
dir_ = '../data/raw/milan/sms-call-internet-mi/sms-call-internet-mi-Nov/sms-call-internet-mi-2013-11-03.txt'
data = pd.read_table(dir_,names = names)
data = data.fillna(0)
d = data.groupby(['squareId','timeInterval']).sum()
d = d.drop(['countryCode'],axis = 1)
str_time = min(d.index.get_level_values(1))
end_time = max(d.index.get_level_values(1))

In [5]:
interaction =  load_npz('demo/1383464400000.npz')

In [9]:
inter = interaction.tocoo()

In [17]:
row = inter.row
col = inter.col
val = inter.data

In [21]:
edges = list(zip(row,col,val))

In [45]:
tmp = 'tmp.txt'
f = open(tmp, "a+")
for r,c,v in edges:
    f.write('{} {} {}\n'.format(r,c,v))
f.close()

### unsupervised embedding
进行简单的低维嵌入，只是简单的利用节点之间的通讯交互

#### deep walk


In [42]:
from libnrl.graph import *
from libnrl import node2vec

In [46]:
g = Graph()
g.read_edgelist(filename='tmp.txt', weighted=True, directed=True)

In [47]:
model = node2vec.Node2vec(graph=g, path_length=10,num_paths=5, dim=10,workers=4, window=5, dw=True)

Walk iteration:
1 / 5
2 / 5
3 / 5
4 / 5
5 / 5
Learning representation...


In [48]:
model.save_embeddings('embedding.txt')

#### forming edges

In [51]:
df1 = pd.read_csv('D:/MobileTrafficPrediction/data/processed/gridTraffic/gridTraffic/grid'+str(1)+'.csv')
df1 = df1.rename(columns={0: 'time', 1: 'sms_in', 2: 'sms_out', 3: 'call_in', 4: 'call_out', 5: "traffic"})

In [52]:
dfcallout = pd.read_csv('D:/MobileTrafficPrediction/data/processed/callout_embedding.csv')

In [54]:
dfcallout

,0,1,2,3,4,5
0,0.440658,-0.680097,0.903580,0.570484,-0.631873,0.628311
1,0.456710,-0.679450,0.905576,0.569363,-0.632157,0.628442
2,0.473418,-0.678763,0.907519,0.568152,-0.632445,0.628567
3,0.393047,-0.681944,0.897081,0.573573,-0.630983,0.627879
4,0.335413,-0.683972,0.888065,0.577134,-0.629987,0.627634
5,0.473418,-0.678763,0.907519,0.568152,-0.632445,0.628567
6,0.473418,-0.678763,0.907519,0.568152,-0.632445,0.628567
7,0.473418,-0.678763,0.907519,0.568152,-0.632445,0.628567
8,0.473418,-0.678763,0.907519,0.568152,-0.632445,0.628567
9,0.265886,-0.686046,0.875567,0.581425,-0.629082,0.628149
